# Practical week 1 - Crash course

MAQ - 32806, Imme Benedict, 2016

This tutorial provides a crash course related to ipython notebooks and the python language itself, which we will both use during this course of Atmospheric Dynamics.

# 1. iPython Notebook

Python notebook is an interactive computational environment, in which you can combine code execution, text, 
mathematics and plots. 

By using cells, different cells can be assigned as either text or code. 
This cell is assigned to text as is indicated by the markdown option above.

If you want to include a formula in the text you can do so by using the latex way of writing an equation, for example:
    $$ \dfrac{v}{t} = 0 $$

To write Python code we have to start a new cell which we assign to code:

In [ ]:
# This cell is assigned to Code

a = 5
b = 6
c = a + b
print c

# 2. Python coding

In this tutorial you learn how to read in a netcdf file and plot certain variables from this file.

In [ ]:
# now we can start with python coding
# python is based on different packages which we upload first

import numpy as np              # Numpy is the fundamental package for scientific computing in Python.
import netCDF4 as nc            # NetCDF is the data format of the meteorological data that we use.
import matplotlib.pyplot as pl  # Matplotlib is a scientific plotting package.
from mpl_toolkits.basemap import Basemap # Import the map plotting interface.

During this course we will study the dynamics by analysing data from a re-analysis dataset. Re-analysis  is a climate or weather model simulation of the past that includes data assimilation of historical observations. 
The dataset used here is called Era-Interim and is produced at the European Centre for Medium Range Forecasting. It is freely available, and provided in netcdf format, which is a common format in climate science. 

We have provided a dataset from ERA-Interim from one particular day. 
You can read in a dataset in python in the following way:    

In [ ]:
nc_file = nc.Dataset("data/test.nc", "r")  # read in netcdf file
lat = nc_file.variables["latitude"][:]     # upload latitude 
lon = nc_file.variables["longitude"][:]    # upload longitude 
p = nc_file.variables["level"][:]*100      # upload level (in mbars = hPa) *100 to transform it into Pa
Temp = nc_file.variables["t"][:,:,:,:]     # upload variable temperature [time, level, latitude, longitude] 

nc_file.close() # close the file

After reading in the different variables we can start to have a look at the data.

a) Find out the domain of the data and how many different levels are present in the dataset.

A good way to have a look at your data is to plot it. The re-analysis data is presented in the netcdf file with a longitude from 0 to 360 degrees. As we are interested in Europe we want the data to be in the format with a longitude from -180 to 180 degrees. 

In [ ]:
# The statement below enforces the plots to be put into this notebook, instead of in their own windows.
%pylab inline
pl.rcParams.update({'font.size': 12})          # Set the standard font size of the plots to 11pt.
pl.rcParams.update({'figure.figsize': [13,6]}) # Set the standard figure size.

t = -1                                        
p_plot = 50000.                                # the pressure level which we want to plot
n = abs(p-p_plot).argmin()                     
# normalize the pressure levels and find the location of the p_plot value in the list of pressure levels

nroll = lon.size//2
lon = np.roll(lon, nroll)                      # np.roll: roll array elements along a given axis. 
lon = np.where(lon>=180., lon-360., lon)       # change longitude to -180 to 180 degrees

T = np.roll(Temp[t,:,:,:], nroll, -1)          # numpy.roll(a, shift, axis=None)

In [ ]:
m = Basemap(llcrnrlon=-180, llcrnrlat=-90, urcrnrlon=180, urcrnrlat=90, projection='mill', resolution='l')
#create a basemap with the latitudes, longitudes and projection you want to plot

# Convert lat/lon to map coordinates
lons, lats = m(*np.meshgrid(lon, lat))         

# plot the variable T with a colorbar
m.pcolormesh(lons, lats, T[n,:,:])           
m.colorbar()                        

# Draw coastlines
m.drawcoastlines(color='#222222')

# Draw parallels and meridians.
m.drawparallels(np.arange(-90.,90.,30.),labels=[1,0,0,0])
m.drawmeridians(np.arange(0.,360.,60.),labels=[0,0,0,1])

pl.title('T at p = {0}'.format(p[n]));         # plot title
pl.tight_layout()

b) Create a map only over Europe by changing the code

c) Plot temperature at the surface

We have now looked at temperature over a horizontal domain. But we can also plot temperature in the vertical.

In [ ]:
# take vertical slice of temperature (at Cabauw 51.9653° N, 4.8979° E )
lon_plot = 4.8979                           
lat_plot = 51.9653
lon_n = abs(lon-lon_plot).argmin()     
lat_n = abs(lat-lat_plot).argmin()

# plot figure
pl.figure()
plot(T[:,lat_n, lon_n],p, 'b', label= 'Cabauw')
pl.xlabel('Temperature [K]')                    # Assign title to x-axis
pl.ylabel('height [Pa]')                        # Assign title to y-axis
# invert yaxis as pressure coordinates go from high to low
pl.gca().invert_yaxis()   
leg1 = legend(loc=1,ncol=1)                     # include legend

d) Include a vertical profile of a different location in the same plot

To asses and bewerk your data often loops are used to assign each element is a list. Here we show an example of a loop in python: We want to calculate the gradient in temperature between the different grid points.

In [ ]:
T_gradient_lon = np.zeros(T[n,:,:].shape)
for i in range(1,lon.size): # T [level, latitude, longitude]
    T_gradient_lon[:,i] =  T[n,:,i] - T[n,:,i-1]
    
T_gradient_lat = np.zeros(T[n,:,:].shape)
for j in range(lat.size):
    T_gradient_lat[j,] = T[n,j,:] - T[n,j-1,:]

Now we have calculated the gradient of the temperature in both the x and the y direction. 
Python also has an automatic function for this called np.gradient. 
np.diff : Calculate the n-th discrete difference along given axis

`T_gradient_lat1 = np.diff(T[n,:,:], axis=0)`
T_gradient_lon1 = np.diff(T[n,:,:], axis=1)

e) Check if both methods have produces the same output.

f) Python also provides the function np.gradient. Find out what happens when you apply this function.